In [1]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.base import backtest_df_only
from quantfreedom.enums import *
from quantfreedom.helper_funcs import create_os_cart_product_nb
from quantfreedom.strategies.strategy_example import StrategyExample

mufex = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False
)
symbol = "BTCUSDT"
%load_ext autoreload
%autoreload 2

In [2]:
candles = mufex.get_candles_df(symbol=symbol, timeframe='5m', candles_to_dl=2000)

It took 00 mins and 03 seconds to download 2000 candles


In [3]:
mufex.set_exchange_settings(
    symbol=symbol,
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings()
order_settings_arrays = OrderSettingsArrays(
    long_or_short=np.array([LongOrShortType.Long]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    risk_account_pct_size=np.array([1]),
    max_equity_risk_pct=np.array([3]),
    stop_loss_type=np.array([StopLossStrategyType.SLBasedOnCandleBody]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    sl_based_on_add_pct=np.array([0.01]),
    sl_based_on_lookback=np.array([20]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]),
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]),
    trail_sl_by_pct=np.array([1.0]),
    take_profit_type=np.array([TakeProfitStrategyType.RiskReward]),
    risk_reward=np.array([3,5]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
    leverage_type=np.array([LeverageStrategyType.Dynamic]),
    static_leverage=np.array([0.0]),
    num_candles=np.array([200]),
    entry_size_asset=np.array([0.0]),
    max_trades=np.array([3]),
    
)
os_cart_arrays = create_os_cart_product_nb(
    order_settings_arrays=order_settings_arrays,
)

In [7]:
strategy = StrategyExample(
    candle_processing_mode=CandleProcessingType.Backtest,
    candles=candles,
    rsi_length=[14],
    rsi_is_below=[50, 80],
)
candles_reg = backtest_df_only(
    starting_equity=1000.0,
    os_cart_arrays=os_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=mufex.exchange_settings,
    strategy=strategy,
    candles=candles,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 2
Total order settings to test: 2
Total combinations of settings to test: 4

Total candles: 2,000
Total candles to test: 8,000


In [8]:
candles_reg.sort_values(['ind_set_idx','or_set_idx']).iloc[:]

,ind_set_idx,or_set_idx,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
0,0,0,163.0,-90.32,9.20,-0.0000,-903.2222,96.7778
1,0,1,159.0,-91.31,9.43,-0.0000,-913.0738,86.9262
2,1,0,164.0,-90.43,8.54,-0.9186,-904.2577,95.7423
3,1,1,160.0,-91.66,8.75,-0.9201,-916.6461,83.3539


In [ ]:
mufex.exchange_settings